In [1]:
import os
from tqdm import tqdm

import pandas as pd
from parse_tool import PDFProcessor
from text_processing import *


In [2]:
# NAMESPACE = 'SANTAFE_MX5_2023'
NAMESPACE = 'TESLA_MODEL3'
# pdf_file = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_parser/pdf/2024 아이오닉 5 (IONIQ 5) NE.pdf'

# pdf_file = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_parser/pdf/디 올 뉴 싼타페 MX5.pdf'
pdf_file = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_parser/pdf/TESLA_MODEL3.pdf'
context_file_root_path = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_context'

os.makedirs(context_file_root_path + f'/image/{NAMESPACE}', exist_ok=True)
os.makedirs(context_file_root_path + f'/csv/{NAMESPACE}', exist_ok=True)
os.makedirs(context_file_root_path + f'/text/{NAMESPACE}', exist_ok=True)
context_text_dir = context_file_root_path + f'/text/{NAMESPACE}'

fitz_page_start = 7
fitz_page_end = 253
HYUNNDAI_y_range = (39, 575)
TESLA_y_range=(39,775)


In [10]:
company = 'hyundai'
if NAMESPACE.startswith("TESLA"):
    company = 'tesla'

In [11]:
pdf_processor = PDFProcessor(NAMESPACE, pdf_file, fitz_page_start, fitz_page_end, y_range=TESLA_y_range, context_file_root_path = context_file_root_path, company=company)

In [12]:
span_list = []
contents_list = []
img_urls = []
csv_urls = []
contents_type = []
block_nums = []
while 1:
    try:
        if pdf_processor.page_num % 50 == 0:
            print(f"Processing page {pdf_processor.page_num}")
        next(pdf_processor)
        page_result = pdf_processor.parse_pdf()
        
        span_list.extend(page_result['span'])
        contents_list.extend(page_result['contents'])
        csv_urls.extend(page_result['csv_urls'])
        img_urls.extend(page_result['img_urls'])
        contents_type.extend(page_result['contents_type'])
        block_nums.extend(page_result['block_num'])
    except StopIteration as e:
        print('End of processing', e)
        break
    except Exception as e:
        print(f"Error raised: {e}, page_num: {pdf_processor.page_num}")
        break

Processing page 50
Processing page 100
Processing page 150
Processing page 200
Processing page 250
End of processing 


In [13]:
df = pd.DataFrame({
    'span': span_list,
    'contents': contents_list,
    'csv_urls': csv_urls,
    'img_urls': img_urls,
    'ctype': contents_type,
    'block_num': block_nums
})

df = df[(df['ctype']=='image') | (~df['contents'].isna())]

In [14]:
df.to_csv(f'{context_text_dir}/{NAMESPACE}_result.csv', index=False)
df.to_parquet(f'{context_text_dir}/{NAMESPACE}_result.parquet', index=False)

# 목차 생성

In [17]:
fitz_page_start = 1
fitz_page_end = 3
pdf_processor = PDFProcessor(NAMESPACE, pdf_file, fitz_page_start, fitz_page_end, y_range=TESLA_y_range, context_file_root_path = context_file_root_path, company='tesla')
span_list = []
contents_list = []
img_urls = []
csv_urls = []
contents_type = []
block_nums = []
while 1:
    try:
        if pdf_processor.page_num % 50 == 0:
            print(f"Processing page {pdf_processor.page_num}")
        next(pdf_processor)
        page_result = pdf_processor.parse_pdf()
        
        span_list.extend(page_result['span'])
        contents_list.extend(page_result['contents'])
        csv_urls.extend(page_result['csv_urls'])
        img_urls.extend(page_result['img_urls'])
        contents_type.extend(page_result['contents_type'])
        block_nums.extend(page_result['block_num'])
    except StopIteration as e:
        print('End of processing', e)
        break
    except Exception as e:
        print(f"Error raised: {e}, page_num: {pdf_processor.page_num}")
        break
    
header_df = pd.DataFrame({
    'span': span_list,
    'contents': contents_list,
    'csv_urls': csv_urls,
    'img_urls': img_urls,
    'ctype': contents_type,
    'block_num': block_nums
})

header_df = header_df[(header_df['ctype']=='image') | (~header_df['contents'].isna())]

End of processing 


In [18]:
header_df

,span,contents,csv_urls,img_urls,ctype,block_num
0,"""('NotoSansKR-Regular', 0, 12)""",개요...............................................,None,None,text,2_0
1,"""('NotoSansKR-Regular', 0, 9)""",외관...............................................,None,None,text,2_0
2,"""('NotoSansKR-Regular', 0, 9)""",실내...............................................,None,None,text,2_0
3,"""('NotoSansKR-Regular', 0, 9)""",터치스크린............................................,None,None,text,2_0
4,"""('NotoSansKR-Regular', 0, 9)""",실내 전자 장치.........................................,None,None,text,2_0
...,...,...,...,...,...,...
104,"""('NotoSansKR-Regular', 0, 9)""",기능 사용 가능 여부 설명...................................,None,None,text,3_3
105,"""('NotoSansKR-Regular', 0, 9)""",면책 조항............................................,None,None,text,3_3
106,"""('NotoSansKR-Regular', 0, 9)""",안전 결함 보고.........................................,None,None,text,3_3
107,"""('NotoSansKR-Regular', 0, 9)""",인증 적합성...........................................,None,None,text,3_3


In [19]:
header_df[['contents', 'page']] = header_df['contents'].apply(lambda x: pd.Series([x.split('.')[0], x.split('.')[-1]]))
header_df['page'] = header_df['page'].astype(int)
header_df['page'] = header_df['page'] + 2
header_df.loc[header_df['span'].astype(str)=='"(\'NotoSansKR-Regular\', 0, 12)"', 'index'] = 'h1'
header_df.loc[header_df['span'].astype(str)=='"(\'NotoSansKR-Regular\', 0, 9)"', 'index'] = 'h2'
header_df.loc[header_df['index'] == 'h1', 'tmp_order'] = -2
header_df.loc[header_df['index'] == 'h2', 'tmp_order'] = -1
header_df['tmp_order'] = header_df['tmp_order'].astype(int)

In [20]:
header_df.loc[header_df['index'] == 'h1', 'block_num'] = header_df.loc[header_df['index'] == 'h1', 'page'].astype(str) + '_0'
header_df.loc[header_df['index'] == 'h2', 'block_num'] = header_df.loc[header_df['index'] == 'h2', 'page'].astype(str) + '_1'

# 텍스트 정제

In [28]:
df = pd.read_parquet(f'{context_text_dir}/{NAMESPACE}_result.parquet')
df[['page', 'num']] = df['block_num'].apply(lambda x: pd.Series([x.split('_')[0], x.split('_')[1]]))
df['block_num'] = df['page'] + '_' + (df['num'].astype(int) + 2).astype(str)
df['page'] = df['page'].astype(int)

In [29]:
df = remove_invalid_row_by_span(df, 'tesla')
df = add_header(df, 'tesla')

In [30]:
df = df.reset_index(names='tmp_order')
df['tmp_order'] = df['tmp_order']

In [31]:
df = pd.concat([header_df, df], axis=0).sort_values(['page', 'tmp_order']).drop(columns = ['tmp_order', 'num'])

In [34]:
block_df = group_by_block(df)
doc_df = group_by_doc(block_df)


In [38]:
doc_df[doc_df['h3'].str.contains('충전 중지')]['doc_contents'].values[0]

'충전 중지\n\n충전 중 언제든지 충전 케이블의 연결을 해제하거나 터치스크린에서 충전 중지를 터치하여 충전을 중지하십시오.\n\n참고: 승인 없이 충전 케이블 연결을 분리할 수 없도록 충전 케이블 래치가 잠겨 있어서 Model 3이(가) 잠금 해제되거나 승인된 스마트폰을 인식한 이후에만 충전 케이블의 연결을 분리할 수 있습니다.\n\n참고: 초기 세대 충전 포트가 장착된 경우 충전 포트는 Model 3이(가) 잠겨 있는 경우에도 차량이 충전 중이 아닐 때와 주변 온도가 41°F(5°C) 미만으로 낮을 때 잠금 해제된 상태로 유지됩니다.\n\n다음과 같이 충전 케이블 연결을 분리합니다.\n\n1. 커넥터 핸들의 버튼을 길게 눌러 래치를 해제합니다.\n\n2. 충전 포트에서 커넥터를 당깁니다. 충전 포트 도어가 자동으로 닫힙니다.\n\n참고: 다음과 같은 방법을 사용하여 충전 포트 도어를 닫을 수도 있습니다.\n\n• 터치스크린에서 차량 상태 개요의 충전 포트 아이콘(번개)을 터치합니다.\n\n• 터치스크린에서 컨트롤 > 충전 > 충전 포트 닫기로 이동합니다.\n\n• 음성 명령을 사용하여 충전 포트 도어를 닫습니다(음성 명령 페이\n\n지의 19 참조).\n\n경고: 충전 포트 도어를 절대 수동으로 닫지 마세요. 그런 경우 손상을 입힐 수 있습니다.\n\n공용 충전 스테이션에서 어댑터를 사용할 때 충전 케이블을 분리하려면 다음과 같이 합니다.\n\n1. Model 3을(를) 잠금 해제합니다.\n\n2. 공용 충전 핸들을 한 손으로 잡고 어댑터를 다른 손으로 잡은 다음 공용 충전 핸들의 버튼을 누른 상태를 유지하면서 핸들과 어댑터를 바깥쪽으로 잡아 당겨 동시에 분리합니다.\n\n참고: 충전 스테이션 핸들이 어댑터에서 분리되어 어댑터가 Model 3 안에 있는 경우 터치스크린을 사용하여 충전 포트 잠금을 해제합니다.\n\n3. 충전 핸들의 버튼을 다시 길게 눌러 공용 충전 핸들에서 어댑터를 분리합니다.\n\n참고: 충전 포트에서 커넥터를 제거하면 약 10초 뒤에 

In [39]:
doc_df.to_parquet(f'{context_text_dir}/doc_{NAMESPACE}_result.parquet')
doc_df.to_csv(f'{context_text_dir}/doc_{NAMESPACE}_result.csv')